**QUESTION:
1.Get the hardship index for the community area of the school which has College Enrollment of 4368.**

*Answer will be explored in two ways: Pandas and SQL statement*

In [ ]:
import csv, sqlite3
con=sqlite3.connect("Realworlddata.db")
cur=con.cursor()

import prettytable
prettytable.DEFAULT='DEFAULT'

%load_ext sql
%sql sqlite:///Realworlddata.db

In [ ]:
import pandas as pd
df=pd.read_csv("https://data.cityofchicago.org/resource/9xs2-f89t.csv")
df.to_sql("chireport",con, if_exists="replace",index=False,method='multi')

566

In [ ]:
%sql SELECT * FROM chireport LIMIT 1

 * sqlite:///Realworlddata.db
Done.


school_id,name_of_school,elementary_or_high_school,street_address,city,state,zip_code,phone_number,link_,network_manager,collaborative_name,adequate_yearly_progress_made_,track_schedule,cps_performance_policy_status,cps_performance_policy_level,healthy_schools_certified_,safety_icon_,safety_score,family_involvement_icon,family_involvement_score,environment_icon_,environment_score,instruction_icon_,instruction_score,leaders_icon_,leaders_score_,teachers_icon_,teachers_score,parent_engagement_icon_,parent_engagement_score,parent_environment_icon,parent_environment_score,average_student_attendance,rate_of_misconducts_per_100_students_,average_teacher_attendance,individualized_education_program_compliance_rate_,pk_2_literacy,pk_2_math_,gr3_5_grade_level_math,gr3_5_grade_level_read_,gr3_5_keep_pace_read_,gr3_5_keep_pace_math,gr6_8_grade_level_math,gr6_8_grade_level_read,gr6_8_keep_pace_math,gr6_8_keep_pace_read,gr_8_explore_math,gr_8_explore_read,isat_exceeding_math_,isat_exceeding_reading_,isat_value_add_math,isat_value_add_read,isat_value_add_color_math,isat_value_add_color_read,students_taking_algebra,students_passing_algebra,_th_grade_explore_2009_,_th_grade_explore_2010_,_th_grade_plan_2009_,_th_grade_plan_2010_,net_change_explore_and_plan,_th_grade_average_act_2011_,net_change_plan_and_act,college_eligibility_,graduation_rate_,college_enrollment_rate,college_enrollment_number_of_students_,general_services_route_,freshman_on_track_rate,rcdts_code,x_coordinate,y_coordinate,latitude,longitude,community_area_number,community_area_name,ward,police_district,location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99.0,Very Strong,99,Strong,74.0,Strong,66.0,Strong,65,Strong,70,Strong,56,Average,47,96.0,2.0,96.4,95.8,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,150000000000000,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"


In [ ]:
socio_df=pd.read_csv("https://data.cityofchicago.org/resource/jcxq-k9xf.csv")
socio_df.to_sql("chi_socioeco",con,if_exists='replace',index=False,method="multi")

socio_df.head(2)

,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0


In [ ]:
### To find out if "ca" in socioeconomic datasets equals to "community_area_number" in chicago schools report.
### If found in socio_df and the result is 7, it means the two data fields match.

socio_df[socio_df["community_area_name"]=="Lincoln Park"]

,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
6,7.0,Lincoln Park,0.8,12.3,5.1,3.6,21.5,71551,2.0


In [ ]:
# step 1) find the school match with enrollment=4368

school_match=df.loc[df["college_enrollment_number_of_students_"]==4368,["name_of_school","community_area_name","community_area_number","college_enrollment_number_of_students_"]]
school_match

,name_of_school,community_area_name,community_area_number,college_enrollment_number_of_students_
6,Albert G Lane Technical High School,NORTH CENTER,5,4368


In [ ]:
# step 2) ensure match keys match type (ca is float in socio_df)

socio_df["ca"]=socio_df["ca"].astype("Int64")
df["community_area_number"]=df["community_area_number"].astype("Int64")

In [ ]:
# step 3) merge

result=school_match.merge(socio_df[["ca","hardship_index"]],left_on="community_area_number", right_on="ca",how="left")
result[["name_of_school","community_area_number","hardship_index"]]

,name_of_school,community_area_number,hardship_index
0,Albert G Lane Technical High School,5,6.0


Below is the SQLite way.

In [ ]:
%%sql
SELECT c.name_of_school, c.community_area_name, c.community_area_number,c.college_enrollment_number_of_students_,
s.hardship_index
FROM chireport c
JOIN chi_socioeco s
ON c.community_area_number=CAST(s.ca AS INTEGER)
WHERE c.college_enrollment_number_of_students_=4368;


 * sqlite:///Realworlddata.db
Done.


name_of_school,community_area_name,community_area_number,college_enrollment_number_of_students_,hardship_index
Albert G Lane Technical High School,NORTH CENTER,5,4368,6.0


In [ ]:
%%sql
SELECT c.name_of_school, c.community_area_name, c.community_area_number, c.college_enrollment_number_of_students_
FROM chireport c, chi_socioeco s
WHERE c.college_enrollment_number_of_students_=4368
AND c.community_area_number=CAST(s.ca AS INTERGER);

 * sqlite:///Realworlddata.db
Done.


name_of_school,community_area_name,community_area_number,college_enrollment_number_of_students_
Albert G Lane Technical High School,NORTH CENTER,5,4368


**QUESTION: 2.Get the hardship index for the community area which has the highest value for College Enrollment.**

*Answer will be explored in two ways: Pandas and SQL statement*

In [ ]:
max_enroll=df["college_enrollment_number_of_students_"].max()
school=df[df["college_enrollment_number_of_students_"]==max_enroll]

socio_df["ca"]=socio_df["ca"].astype("Int64")
df["community_area_number"]=df["community_area_number"].astype("Int64")

result=school.merge(socio_df[["ca","hardship_index"]],left_on="community_area_number",right_on="ca",how="left")
result[["name_of_school","community_area_name","community_area_number","college_enrollment_number_of_students_","hardship_index"]]

### keeping the [["ca","hardship_index"]] in the merge formula helps to limit the data needed in df2 better.

,name_of_school,community_area_name,community_area_number,college_enrollment_number_of_students_,hardship_index
0,Albert G Lane Technical High School,NORTH CENTER,5,4368,6.0


In [ ]:
%%sql
SELECT c.name_of_school, c.community_area_name, c.community_area_number,c.college_enrollment_number_of_students_, s.hardship_index
FROM chireport c
JOIN chi_socioeco s
ON c.community_area_number=CAST(s.ca AS INTEGER)
WHERE c.college_enrollment_number_of_students_=(SELECT MAX(college_enrollment_number_of_students_) FROM chireport)

 * sqlite:///Realworlddata.db
Done.


name_of_school,community_area_name,community_area_number,college_enrollment_number_of_students_,hardship_index
Albert G Lane Technical High School,NORTH CENTER,5,4368,6.0
